In [1]:
## See storage available
!df -h

Filesystem       Size   Used  Avail Capacity iused      ifree %iused  Mounted on
/dev/disk3s1s1  460Gi   15Gi  150Gi     9%  575614 1572258520    0%   /
devfs           201Ki  201Ki    0Bi   100%     698          0  100%   /dev
/dev/disk3s6    460Gi   20Ki  150Gi     1%       0 1572258520    0%   /System/Volumes/VM
/dev/disk3s2    460Gi  302Mi  150Gi     1%     749 1572258520    0%   /System/Volumes/Preboot
/dev/disk3s4    460Gi  980Ki  150Gi     1%      40 1572258520    0%   /System/Volumes/Update
/dev/disk1s2    500Mi  6.0Mi  480Mi     2%       1    4914080    0%   /System/Volumes/xarts
/dev/disk1s1    500Mi  7.3Mi  480Mi     2%      30    4914080    0%   /System/Volumes/iSCPreboot
/dev/disk1s3    500Mi  2.0Mi  480Mi     1%      49    4914080    0%   /System/Volumes/Hardware
/dev/disk3s5    460Gi  295Gi  150Gi    67%  875280 1572258520    0%   /System/Volumes/Data
map auto_home     0Bi    0Bi    0Bi   100%       0          0  100%   /System/Volumes/Data/home


In [2]:
# See CPUs available (mac)
!sysctl -a | grep machdep.cpu

machdep.cpu.brand_string: Apple M1 Pro
machdep.cpu.core_count: 8
machdep.cpu.cores_per_package: 8
machdep.cpu.logical_per_package: 8
machdep.cpu.thread_count: 8


In [3]:
# See memory available (mac)
!vm_stat | perl -ne '/page size of (\d+)/ and $size=$1; /Pages\s+([^:]+)[^\d]+(\d+)/ and printf("%-16s % 16.2f Mi\n", "$1:", $2 * $size / 1048576);'

free:                     1286.69 Mi
active:                   3687.62 Mi
inactive:                 5377.00 Mi
speculative:              2831.34 Mi
throttled:                   0.00 Mi
wired down:               1249.98 Mi
purgeable:                  16.67 Mi
copy-on-write:            4259.81 Mi
zero filled:             65107.00 Mi
reactivated:              7779.62 Mi
purged:                    306.25 Mi
stored in compressor:          3405.31 Mi
occupied by compressor:          1336.22 Mi


In [4]:
%load_ext autoreload
%autoreload 2

# MISC
import gzip
import multiprocessing
import time
import json
from collections import defaultdict
cpu_cores = multiprocessing.cpu_count()
data_path_50MB = '/Users/angelloparr/Documents/csc369/project.nosync/data_uncompressed/Gift_Cards.json'
data_path_263MB = '/Users/angelloparr/Documents/csc369/project.nosync/data_uncompressed/Appliances.json'
data_path_755MB = '/Users/angelloparr/Documents/csc369/project.nosync/data_uncompressed/Digital_Music.json'
data_path_2GB = '/Users/angelloparr/Documents/csc369/project.nosync/data_uncompressed/Video_Games.json'
data_path_10GB = '/Users/angelloparr/Documents/csc369/project.nosync/data_uncompressed/reviews_Books_1.json'
data_path_38GB = '/Users/angelloparr/Documents/csc369/project.nosync/data_uncompressed/Books_big.json'


# NLTK
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Dask
import pandas as pd
import numpy as np
import dask
from dask import dataframe as dd
import dask.bag as db
dask.config.set(scheduler='processes', num_workers=cpu_cores)

# Spark
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql.functions import udf

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/10 17:08:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
def time_it(func, args):
    start = time.time()
    res = func(args)
    elapsed = time.time() - start
    log_line = f'Elapsed time for {func}: {elapsed}'
    return {'res': res, 'elapsed': elapsed, 'log': log_line}

def get_elapsed(start):
    return time.time() - start

def print_elapsed(start):
    elapsed = time.time() - start
    print(f'Elapsed time for operation: {elapsed}')

In [7]:
reviews_df = pd.read_json(data_path_50MB, orient='records', lines=True)
reviews_df.head(5)

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,1,25,False,"12 19, 2008",APV13CM0919JD,B001GXRQW0,{'Gift Amount:': ' 50'},LEH,"Amazon,\nI am shopping for Amazon.com gift car...",Merry Christmas.,1229644800,NaN
1,5,NaN,False,"12 17, 2008",A3G8U1G1V082SN,B001GXRQW0,{'Gift Amount:': ' 50'},Tali,"I got this gift card from a friend, and it was...",Gift card with best selection,1229472000,NaN
2,5,4,False,"12 17, 2008",A11T2Q0EVTUWP,B001GXRQW0,{'Gift Amount:': ' 50'},Z,aren't we going to save trees?! :) People who ...,A convenient and great gift for the environmen...,1229472000,NaN
3,5,NaN,False,"12 17, 2008",A9YKGBH3SV22C,B001GXRQW0,{'Gift Amount:': ' 25'},Giotravels,You can always get someone something from Amaz...,Totally make sense,1229472000,NaN
4,1,NaN,True,"12 17, 2008",A34WZIHVF3OKOL,B001GXRQW0,NaN,King Dad,Why take 50 dollars of good money with no limi...,Give CASH!,1229472000,NaN


## Pandas Operations

In [8]:
begin = time.time()
start = begin

## Drop unwanted columns / keep good ones
columns = ['reviewerID', 'reviewText']
reviews_df = reviews_df[columns]
print_elapsed(start)

## Drop empty (NaN) rows
start = time.time()
reviews_df = reviews_df.dropna()
print_elapsed(start)

## Tokenize words in review
start = time.time()
reviews_df['tokenized_dirty'] = reviews_df['reviewText'].apply(nltk.word_tokenize)
print_elapsed(start)

## Remove stop words
start = time.time()
reviews_df['tokenized_review'] = reviews_df['tokenized_dirty'].apply(lambda tokens: [w for w in tokens if w.lower() not in stop_words])

## Perform Part-Of-Speech (POS) tagging on tokens
def my_pos_tag(tok_list):
    pos_tagged = nltk.pos_tag(tok_list)
    tags = {
        'n' : wordnet.NOUN,
        'v' : wordnet.VERB,
        'j' : wordnet.ADJ,
        'r' : wordnet.ADV
    }
    tag_dict = defaultdict(lambda: wordnet.NOUN, tags)
    new_tags = [(tok, tag_dict[pos[0].lower()]) for (tok, pos) in pos_tagged]
    return new_tags

start = time.time()
reviews_df['tok/pos'] = reviews_df['tokenized_review'].apply(my_pos_tag)
print_elapsed(start)

## Stem words in tokenized review
start = time.time()
reviews_df['stemmed_review'] = reviews_df['tokenized_review'].apply(lambda tokens: [stemmer.stem(t) for t in tokens])
print_elapsed(start)

## Lemmatize words in tokenized review
start = time.time()
reviews_df['lemmatized_review'] = reviews_df['tok/pos'].apply(lambda tokens: [lemmatizer.lemmatize(t, pos) for (t, pos) in tokens])
print_elapsed(start)

print(f'Total execution time: {time.time() - begin}')


Elapsed time for operation: 0.003943920135498047
Elapsed time for operation: 0.02343606948852539
Elapsed time for operation: 13.133556842803955
Elapsed time for operation: 47.215304136276245
Elapsed time for operation: 14.041398048400879
Elapsed time for operation: 3.146782159805298
Total execution time: 77.99114394187927


In [9]:
reviews_df

,reviewerID,reviewText,tokenized_dirty,tokenized_review,tok/pos,stemmed_review,lemmatized_review
0,APV13CM0919JD,"Amazon,\nI am shopping for Amazon.com gift car...","[Amazon, ,, I, am, shopping, for, Amazon.com, ...","[Amazon, ,, shopping, Amazon.com, gift, cards,...","[(Amazon, n), (,, n), (shopping, v), (Amazon.c...","[amazon, ,, shop, amazon.com, gift, card, chri...","[Amazon, ,, shop, Amazon.com, gift, card, Chri..."
1,A3G8U1G1V082SN,"I got this gift card from a friend, and it was...","[I, got, this, gift, card, from, a, friend, ,,...","[got, gift, card, friend, ,, best, !, site, mu...","[(got, v), (gift, a), (card, n), (friend, n), ...","[got, gift, card, friend, ,, best, !, site, mu...","[get, gift, card, friend, ,, best, !, site, mu..."
2,A11T2Q0EVTUWP,aren't we going to save trees?! :) People who ...,"[are, n't, we, going, to, save, trees, ?, !, :...","[n't, going, save, trees, ?, !, :, ), People, ...","[(n't, r), (going, v), (save, v), (trees, n), ...","[n't, go, save, tree, ?, !, :, ), peopl, compl...","[n't, go, save, tree, ?, !, :, ), People, comp..."
3,A9YKGBH3SV22C,You can always get someone something from Amaz...,"[You, can, always, get, someone, something, fr...","[always, get, someone, something, Amazon, safe...","[(always, r), (get, v), (someone, n), (somethi...","[alway, get, someon, someth, amazon, safeti, n...","[always, get, someone, something, Amazon, safe..."
4,A34WZIHVF3OKOL,Why take 50 dollars of good money with no limi...,"[Why, take, 50, dollars, of, good, money, with...","[take, 50, dollars, good, money, limitations, ...","[(take, v), (50, n), (dollars, n), (good, a), ...","[take, 50, dollar, good, money, limit, ,, turn...","[take, 50, dollar, good, money, limitation, ,,..."
...,...,...,...,...,...,...,...
147189,A2K9WVQW9TLWNK,I always enjoy getting these Gift cards via em...,"[I, always, enjoy, getting, these, Gift, cards...","[always, enjoy, getting, Gift, cards, via, ema...","[(always, r), (enjoy, v), (getting, v), (Gift,...","[alway, enjoy, get, gift, card, via, email, lo...","[always, enjoy, get, Gift, card, via, email, L..."
147190,A149ALSR6TPGF7,Worked great,"[Worked, great]","[Worked, great]","[(Worked, v), (great, a)]","[work, great]","[Worked, great]"
147191,A2Q066NZCQSCOR,Gift card,"[Gift, card]","[Gift, card]","[(Gift, n), (card, n)]","[gift, card]","[Gift, card]"
147192,A1KJLWCW7XBS8I,"What is there to say, It's a gift card.","[What, is, there, to, say, ,, It, 's, a, gift,...","[say, ,, 's, gift, card, .]","[(say, v), (,, n), ('s, n), (gift, n), (card, ...","[say, ,, 's, gift, card, .]","[say, ,, 's, gift, card, .]"


## Dask Operations

In [12]:
begin = time.time()
start = begin

# reviews_dd = dd.read_json(data_path_755MB, orient='records')
# reviews_dd = reviews_dd.repartition(npartitions=500)

reviews_dd = db.read_text(data_path_50MB).map(json.loads).to_dataframe()
reviews_dd = reviews_dd.repartition(npartitions=8)

print_elapsed(start)

## Drop unwanted columns / keep good ones
start = time.time()
columns = ['reviewerID', 'reviewText']
reviews_dd = reviews_dd[columns]
print_elapsed(start)

## Drop empty (NaN) rows
start = time.time()
reviews_dd = reviews_dd.dropna()
print_elapsed(start)

## Tokenize words in review
start = time.time()
reviews_dd['tokenized_dirty'] = reviews_dd['reviewText'].apply(nltk.word_tokenize, meta=('reviewText', 'object'))
print_elapsed(start)

## Remove stop words
start = time.time()
reviews_dd['tokenized_review'] = reviews_dd['tokenized_dirty'].apply(lambda tokens: [w for w in tokens if w.lower() not in stop_words], meta=('tokenized_dirty', 'object'))


## Perform Part-Of-Speech (POS) tagging on tokens
def my_pos_tag(tok_list):
    pos_tagged = nltk.pos_tag(tok_list)
    tags = {
        'n' : wordnet.NOUN,
        'v' : wordnet.VERB,
        'j' : wordnet.ADJ,
        'r' : wordnet.ADV
    }
    tag_dict = defaultdict(lambda: wordnet.NOUN, tags)
    new_tags = [(tok, tag_dict[pos[0].lower()]) for (tok, pos) in pos_tagged]
    return new_tags

start = time.time()
reviews_dd['tok/pos'] = reviews_dd['tokenized_review'].apply(my_pos_tag, meta=('tok/pos', 'object'))
print_elapsed(start)

## Stem words in tokenized review
start = time.time()
reviews_dd['stemmed_review'] = reviews_dd['tokenized_review'].apply(lambda tokens: [stemmer.stem(t) for t in tokens], meta=('tokenized_review', 'object'))
print_elapsed(start)

## Lemmatize words in tokenized review
start = time.time()
reviews_dd['lemmatized_review'] = reviews_dd['tok/pos'].apply(lambda tokens: [lemmatizer.lemmatize(t, pos) for (t, pos) in tokens], meta=('tok/pos', 'object'))
print_elapsed(start)

final_dd = reviews_dd.head(5)

print(f'Total execution time: {time.time() - begin}')


Elapsed time for operation: 0.12734389305114746
Elapsed time for operation: 0.0007479190826416016
Elapsed time for operation: 0.001725912094116211
Elapsed time for operation: 0.0012979507446289062
Elapsed time for operation: 0.1732158660888672
Elapsed time for operation: 0.0015239715576171875
Elapsed time for operation: 0.0014340877532958984
Total execution time: 14.011240005493164


In [13]:
final_dd.head(5)

,reviewerID,reviewText,tokenized_dirty,tokenized_review,tok/pos,stemmed_review,lemmatized_review
0,APV13CM0919JD,"Amazon,\nI am shopping for Amazon.com gift car...","[Amazon, ,, I, am, shopping, for, Amazon.com, ...","[Amazon, ,, shopping, Amazon.com, gift, cards,...","[(Amazon, n), (,, n), (shopping, v), (Amazon.c...","[amazon, ,, shop, amazon.com, gift, card, chri...","[Amazon, ,, shop, Amazon.com, gift, card, Chri..."
1,A3G8U1G1V082SN,"I got this gift card from a friend, and it was...","[I, got, this, gift, card, from, a, friend, ,,...","[got, gift, card, friend, ,, best, !, site, mu...","[(got, v), (gift, a), (card, n), (friend, n), ...","[got, gift, card, friend, ,, best, !, site, mu...","[get, gift, card, friend, ,, best, !, site, mu..."
2,A11T2Q0EVTUWP,aren't we going to save trees?! :) People who ...,"[are, n't, we, going, to, save, trees, ?, !, :...","[n't, going, save, trees, ?, !, :, ), People, ...","[(n't, r), (going, v), (save, v), (trees, n), ...","[n't, go, save, tree, ?, !, :, ), peopl, compl...","[n't, go, save, tree, ?, !, :, ), People, comp..."
3,A9YKGBH3SV22C,You can always get someone something from Amaz...,"[You, can, always, get, someone, something, fr...","[always, get, someone, something, Amazon, safe...","[(always, r), (get, v), (someone, n), (somethi...","[alway, get, someon, someth, amazon, safeti, n...","[always, get, someone, something, Amazon, safe..."
4,A34WZIHVF3OKOL,Why take 50 dollars of good money with no limi...,"[Why, take, 50, dollars, of, good, money, with...","[take, 50, dollars, good, money, limitations, ...","[(take, v), (50, n), (dollars, n), (good, a), ...","[take, 50, dollar, good, money, limit, ,, turn...","[take, 50, dollar, good, money, limitation, ,,..."


## Spark Operations

In [15]:
begin = time.time()
start = begin

spark_df = spark.read.json(data_path_50MB)
print_elapsed(start)

## Drop unwanted columns / keep good ones
start = time.time()
columns = ['reviewerID', 'reviewText']
spark_df = spark_df.select(*columns)
print_elapsed(start)

## Drop empty (NaN) rows
start = time.time()
spark_df = spark_df.dropna()
print_elapsed(start)

## Tokenize words in review
start = time.time()
udf_tokenize = udf(nltk.word_tokenize)
spark_df = spark_df.withColumn("tokenized_dirty", udf_tokenize(spark_df.reviewText))
print_elapsed(start)

## Remove stop words in reviews
start = time.time()
udf_clean = udf(lambda tokens: [w for w in tokens if w.lower() not in stop_words])
spark_df = spark_df.withColumn("tokenized_review", udf_clean(spark_df.tokenized_dirty))

## Perform Part-Of-Speech (POS) tagging on tokens
def my_pos_tag(tok_list):
    pos_tagged = nltk.pos_tag(tok_list)
    tags = {
        'n' : wordnet.NOUN,
        'v' : wordnet.VERB,
        'j' : wordnet.ADJ,
        'r' : wordnet.ADV
    }
    tag_dict = defaultdict(lambda: wordnet.NOUN, tags)
    new_tags = [(tok, tag_dict[pos[0].lower()]) for (tok, pos) in pos_tagged]
    return new_tags

start = time.time()
udf_pos = udf(my_pos_tag)
spark_df = spark_df.withColumn('tok_pos', udf_pos(spark_df.tokenized_review))
print_elapsed(start)

## Stem words in tokenized review
start = time.time()
udf_stem = udf(lambda tokens: [stemmer.stem(t) for t in tokens])
spark_df = spark_df.withColumn("stemmed_text", udf_stem(spark_df.tokenized_review))
print_elapsed(start)

## Lemmatize words in tokenized review
start = time.time()
udf_lemmatize = udf(lambda tokens: [lemmatizer.lemmatize(t, pos) for (t, pos) in tokens])
spark_df = spark_df.withColumn("lemmatized_text", udf_lemmatize(spark_df.tok_pos))
print_elapsed(start)
   
final_df = spark_df.head(5)

print(f'Total execution time: {time.time() - begin}')


Elapsed time for operation: 6.844664096832275
Elapsed time for operation: 0.050984859466552734
Elapsed time for operation: 0.042524099349975586
Elapsed time for operation: 0.053797006607055664
Elapsed time for operation: 0.13108396530151367
Elapsed time for operation: 0.025659799575805664
Elapsed time for operation: 0.03692007064819336


Total execution time: 9.849095106124878
